In [ ]:
# Standard scientific computing imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# COBRApy for metabolic modeling
import cobra

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")
print(f"COBRApy version: {cobra.__version__}")


In [ ]:
# Load the iJO1366 model
print("Loading iJO1366 metabolic model...")
model = cobra.io.read_sbml_model("bigg_models/iJO1366.xml")

print(f"✅ Model loaded successfully!")
print(f"   Genes: {len(model.genes)}")
print(f"   Reactions: {len(model.reactions)}")
print(f"   Metabolites: {len(model.metabolites)}")
print(f"   Objective: {model.objective.expression}")


In [ ]:
def setup_medium_conditions(model, glucose_uptake=0, acetate_uptake=0, oxygen_uptake=-20):
    """
    Setup medium conditions for the metabolic model.
    
    Parameters:
    -----------
    model : cobra.Model
        The metabolic model to configure
    glucose_uptake : float
        Glucose uptake rate (negative value)
    acetate_uptake : float
        Acetate uptake rate (negative value)
    oxygen_uptake : float
        Oxygen uptake rate (negative value)
    """
    
    # Get exchange reactions
    glucose_rxn = model.reactions.get_by_id('EX_glc__D_e')
    acetate_rxn = model.reactions.get_by_id('EX_ac_e')
    oxygen_rxn = model.reactions.get_by_id('EX_o2_e')
    
    # Set bounds
    glucose_rxn.bounds = (glucose_uptake, 0)
    acetate_rxn.bounds = (acetate_uptake, 0)
    oxygen_rxn.bounds = (oxygen_uptake, 0)
    
    # Allow essential nutrients
    essential_exchanges = {
        'EX_nh4_e': (-1000, 0),    # Ammonium
        'EX_pi_e': (-1000, 0),     # Phosphate
        'EX_so4_e': (-1000, 0),    # Sulfate
        'EX_k_e': (-1000, 0),      # Potassium
        'EX_na1_e': (-1000, 0),    # Sodium
        'EX_mg2_e': (-1000, 0),    # Magnesium
        'EX_ca2_e': (-1000, 0),    # Calcium
        'EX_fe2_e': (-1000, 0),    # Iron
        'EX_fe3_e': (-1000, 0),    # Iron
        'EX_cl_e': (-1000, 0),     # Chloride
        'EX_co2_e': (0, 1000),     # CO2 production
        'EX_h2o_e': (-1000, 1000), # Water
        'EX_h_e': (-1000, 1000),   # Protons
    }
    
    for exchange_id, bounds in essential_exchanges.items():
        try:
            rxn = model.reactions.get_by_id(exchange_id)
            rxn.bounds = bounds
        except:
            pass
    
    print(f"✅ Medium conditions set:")
    print(f"   Glucose uptake: {glucose_uptake} mmol/gDW/h")
    print(f"   Acetate uptake: {acetate_uptake} mmol/gDW/h")
    print(f"   Oxygen uptake: {oxygen_uptake} mmol/gDW/h")


In [ ]:
def generate_phpp_data(model, substrate1_range, substrate2_range, substrate1_id, substrate2_id):
    """
    Generate phenotype phase plane data.
    
    Parameters:
    -----------
    model : cobra.Model
        The metabolic model
    substrate1_range : array-like
        Range of values for first substrate
    substrate2_range : array-like
        Range of values for second substrate
    substrate1_id : str
        ID of first substrate exchange reaction
    substrate2_id : str
        ID of second substrate exchange reaction
    
    Returns:
    --------
    tuple : (X, Y, Z) arrays for plotting
    """
    
    X, Y = np.meshgrid(substrate1_range, substrate2_range)
    Z = np.zeros_like(X)
    
    # Get exchange reactions
    rxn1 = model.reactions.get_by_id(substrate1_id)
    rxn2 = model.reactions.get_by_id(substrate2_id)
    
    print(f"Generating PhPP data for {substrate1_id} vs {substrate2_id}...")
    
    for i, val1 in enumerate(substrate1_range):
        for j, val2 in enumerate(substrate2_range):
            # Set bounds
            rxn1.bounds = (val1, 0)
            rxn2.bounds = (val2, 0)
            
            # Optimize
            try:
                solution = model.optimize()
                if solution.status == 'optimal':
                    Z[j, i] = solution.objective_value
                else:
                    Z[j, i] = 0
            except:
                Z[j, i] = 0
    
    return X, Y, Z


In [ ]:
# Setup initial medium conditions
setup_medium_conditions(model)

# Define ranges for glucose vs oxygen analysis
glucose_range = np.linspace(-15, 0, 31)  # -15 to 0 mmol/gDW/h
oxygen_range = np.linspace(-25, 0, 26)   # -25 to 0 mmol/gDW/h

# Generate PhPP data
X_glc_o2, Y_glc_o2, Z_glc_o2 = generate_phpp_data(
    model, glucose_range, oxygen_range, 'EX_glc__D_e', 'EX_o2_e'
)


In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Contour plot
contour = ax1.contourf(X_glc_o2, Y_glc_o2, Z_glc_o2, levels=20, cmap='viridis')
ax1.contour(X_glc_o2, Y_glc_o2, Z_glc_o2, levels=10, colors='white', alpha=0.5, linewidths=0.5)
ax1.set_xlabel('Glucose Uptake (mmol/gDW/h)')
ax1.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax1.set_title('Glucose vs Oxygen Phenotype Phase Plane')
plt.colorbar(contour, ax=ax1, label='Growth Rate (1/h)')

# 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_surface(X_glc_o2, Y_glc_o2, Z_glc_o2, cmap='viridis', alpha=0.8)
ax2.set_xlabel('Glucose Uptake (mmol/gDW/h)')
ax2.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax2.set_zlabel('Growth Rate (1/h)')
ax2.set_title('3D Growth Surface')

plt.tight_layout()
plt.show()

# Print key statistics
print(f"\n📊 Glucose vs Oxygen PhPP Statistics:")
print(f"   Maximum growth rate: {Z_glc_o2.max():.3f} 1/h")
print(f"   Optimal glucose uptake: {glucose_range[Z_glc_o2.argmax() // Z_glc_o2.shape[1]]:.1f} mmol/gDW/h")
print(f"   Optimal oxygen uptake: {oxygen_range[Z_glc_o2.argmax() % Z_glc_o2.shape[0]]:.1f} mmol/gDW/h")


In [ ]:
# Define ranges for acetate vs oxygen analysis
acetate_range = np.linspace(-10, 0, 21)  # -10 to 0 mmol/gDW/h
oxygen_range_acetate = np.linspace(-20, 0, 21)  # -20 to 0 mmol/gDW/h

# Generate PhPP data
X_ac_o2, Y_ac_o2, Z_ac_o2 = generate_phpp_data(
    model, acetate_range, oxygen_range_acetate, 'EX_ac_e', 'EX_o2_e'
)


In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Contour plot
contour = ax1.contourf(X_ac_o2, Y_ac_o2, Z_ac_o2, levels=20, cmap='plasma')
ax1.contour(X_ac_o2, Y_ac_o2, Z_ac_o2, levels=10, colors='white', alpha=0.5, linewidths=0.5)
ax1.set_xlabel('Acetate Uptake (mmol/gDW/h)')
ax1.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax1.set_title('Acetate vs Oxygen Phenotype Phase Plane')
plt.colorbar(contour, ax=ax1, label='Growth Rate (1/h)')

# 3D surface plot
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_surface(X_ac_o2, Y_ac_o2, Z_ac_o2, cmap='plasma', alpha=0.8)
ax2.set_xlabel('Acetate Uptake (mmol/gDW/h)')
ax2.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax2.set_zlabel('Growth Rate (1/h)')
ax2.set_title('3D Growth Surface')

plt.tight_layout()
plt.show()

# Print key statistics
print(f"\n📊 Acetate vs Oxygen PhPP Statistics:")
print(f"   Maximum growth rate: {Z_ac_o2.max():.3f} 1/h")
print(f"   Optimal acetate uptake: {acetate_range[Z_ac_o2.argmax() // Z_ac_o2.shape[1]]:.1f} mmol/gDW/h")
print(f"   Optimal oxygen uptake: {oxygen_range_acetate[Z_ac_o2.argmax() % Z_ac_o2.shape[0]]:.1f} mmol/gDW/h")


In [ ]:
def find_line_of_optimality(X, Y, Z, substrate_name):
    """
    Find the line of optimality for a given substrate.
    
    Parameters:
    -----------
    X, Y, Z : arrays
        PhPP data arrays
    substrate_name : str
        Name of the substrate for labeling
    
    Returns:
    --------
    tuple : (optimal_x, optimal_y) arrays
    """
    
    # Find maximum growth rate for each substrate value
    optimal_x = []
    optimal_y = []
    
    for i in range(X.shape[1]):
        # Find maximum growth rate for this substrate value
        max_idx = np.argmax(Z[:, i])
        optimal_x.append(X[max_idx, i])
        optimal_y.append(Y[max_idx, i])
    
    return np.array(optimal_x), np.array(optimal_y)

# Find lines of optimality
glc_opt_x, glc_opt_y = find_line_of_optimality(X_glc_o2, Y_glc_o2, Z_glc_o2, 'Glucose')
ac_opt_x, ac_opt_y = find_line_of_optimality(X_ac_o2, Y_ac_o2, Z_ac_o2, 'Acetate')

# Plot lines of optimality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Glucose vs Oxygen
contour1 = ax1.contourf(X_glc_o2, Y_glc_o2, Z_glc_o2, levels=20, cmap='viridis')
ax1.plot(glc_opt_x, glc_opt_y, 'r-', linewidth=3, label='Line of Optimality')
ax1.set_xlabel('Glucose Uptake (mmol/gDW/h)')
ax1.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax1.set_title('Glucose vs Oxygen with Line of Optimality')
ax1.legend()
plt.colorbar(contour1, ax=ax1, label='Growth Rate (1/h)')

# Acetate vs Oxygen
contour2 = ax2.contourf(X_ac_o2, Y_ac_o2, Z_ac_o2, levels=20, cmap='plasma')
ax2.plot(ac_opt_x, ac_opt_y, 'r-', linewidth=3, label='Line of Optimality')
ax2.set_xlabel('Acetate Uptake (mmol/gDW/h)')
ax2.set_ylabel('Oxygen Uptake (mmol/gDW/h)')
ax2.set_title('Acetate vs Oxygen with Line of Optimality')
ax2.legend()
plt.colorbar(contour2, ax=ax2, label='Growth Rate (1/h)')

plt.tight_layout()
plt.show()

print("\n📈 Lines of Optimality Analysis:")
print("   The red lines show the optimal oxygen uptake for each substrate uptake rate.")
print("   These represent the most efficient metabolic strategies.")


In [ ]:
# Compare maximum growth rates
max_glucose_growth = Z_glc_o2.max()
max_acetate_growth = Z_ac_o2.max()

# Create comparison plot
fig, ax = plt.subplots(figsize=(10, 6))

substrates = ['Glucose', 'Acetate']
max_growths = [max_glucose_growth, max_acetate_growth]
colors = ['#2E86AB', '#A23B72']

bars = ax.bar(substrates, max_growths, color=colors, alpha=0.8)
ax.set_ylabel('Maximum Growth Rate (1/h)')
ax.set_title('Maximum Growth Rate Comparison')
ax.grid(True, alpha=0.3)

# Add value labels on bars
for bar, growth in zip(bars, max_growths):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{growth:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n📊 Growth Rate Comparison:")
print(f"   Glucose maximum growth: {max_glucose_growth:.3f} 1/h")
print(f"   Acetate maximum growth: {max_acetate_growth:.3f} 1/h")
print(f"   Glucose is {max_glucose_growth/max_acetate_growth:.1f}x more efficient than acetate")
